In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

base_url = 'https://bina.az/alqi-satqi'
page_urls = [f'{base_url}?page={page}' for page in range(1, 1000)]

data = []
for url in page_urls:
    html = requests.get(url)
    x = BeautifulSoup(html.content, 'html.parser')
    link = []
    for i in x.find_all('div', class_='items-i'):
        link.append('https://bina.az/' + i.find('a').get('href'))

    linkk = []
    for i in x.find_all('div', class_='location'):
        linkk.append(i.text)

    room = []
    for i in x.find_all('ul', class_='name'):
        room.append(i.text)

    rooms = []
    area = []
    floor = []
    for r in room:
        r_rooms = re.findall('([0-9].[a-z]{3,4})', r)
        r_area = re.findall(r'(\d+ m²)', r)
        r_floor = re.findall(r'(\d+/\d+)\s*mərtəbə', r)
        rooms.extend(r_rooms)
        area.extend(r_area)
        floor.extend(r_floor)

    list_data = []
    for i in x.find_all('div', class_='city_when'):
        list_data.append(i.text[-11:])

    price = []
    for i in x.find_all('div', class_='abs_block'):
        price.append(i.text)

    min_length = min(len(linkk), len(link), len(list_data), len(price), len(rooms), len(floor), len(area))

    df = pd.DataFrame({
        'Unvan': linkk[:min_length],
        'Link': link[:min_length],
        'Tarix': list_data[:min_length],
        'Qiymet': price[:min_length],
        'rooms': rooms[:min_length],
        'floor': floor[:min_length],
        'area': area[:min_length]
    })

    data.append(df)

final_df = pd.concat(data, ignore_index=True)


KeyboardInterrupt



In [227]:
import pandas as pd
data=pd.read_csv('final.csv')
df=pd.DataFrame(data)
df

,Unnamed: 0.1,Unnamed: 0,Unvan,Link,Tarix,Qiymet,rooms,floor,area
0,0,0,Şağan q.,https://bina.az//items/3495797,bugün 16:06,260 000AZN,3 otaq,13/16,160 m²
1,1,1,Mərdəkan q.,https://bina.az//items/3004266,bugün 16:06,195 000AZN,2 otaq,10/16,160 m²
2,2,2,Neftçilər m.,https://bina.az//items/3632290,bugün 16:05,137 000AZN,2 otaq,7/9,75 m²
3,3,3,Masazır q.,https://bina.az//items/3562385,bugün 16:05,36 000AZN,3 otaq,7/12,60 m²
4,4,4,Qara Qarayev m.,https://bina.az//items/3565113,bugün 16:05,138 600AZN,3 otaq,10/17,5 m²
...,...,...,...,...,...,...,...,...,...
17209,17209,17209,Yasamal r.,https://bina.az//items/3608907,i̇yul 2023,185 000AZN,3 otaq,7/7,400 m²
17210,17210,17210,Badamdar q.,https://bina.az//items/2764090,i̇yul 2023,6 000 000AZN,9 sot,15/18,1500 m²
17211,17211,17211,Bakıxanov q.,https://bina.az//items/3608900,i̇yul 2023,240 000AZN,8 sot,3/9,100 m²
17212,17212,17212,Həzi Aslanov m.,https://bina.az//items/3608859,i̇yul 2023,130 000AZN,3 otaq,9/9,500 m²


In [228]:
df.dtypes

Unnamed: 0.1     int64
Unnamed: 0       int64
Unvan           object
Link            object
Tarix           object
Qiymet          object
rooms           object
floor           object
area            object
dtype: object

In [229]:
df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)


In [230]:
df['Qiymet'] = df['Qiymet'].str.replace('AZN', '').str.replace(' ', '').str.strip()
df['Qiymet'] = df['Qiymet'].astype(int)
df

,Unvan,Link,Tarix,Qiymet,rooms,floor,area
0,Şağan q.,https://bina.az//items/3495797,bugün 16:06,260000,3 otaq,13/16,160 m²
1,Mərdəkan q.,https://bina.az//items/3004266,bugün 16:06,195000,2 otaq,10/16,160 m²
2,Neftçilər m.,https://bina.az//items/3632290,bugün 16:05,137000,2 otaq,7/9,75 m²
3,Masazır q.,https://bina.az//items/3562385,bugün 16:05,36000,3 otaq,7/12,60 m²
4,Qara Qarayev m.,https://bina.az//items/3565113,bugün 16:05,138600,3 otaq,10/17,5 m²
...,...,...,...,...,...,...,...
17209,Yasamal r.,https://bina.az//items/3608907,i̇yul 2023,185000,3 otaq,7/7,400 m²
17210,Badamdar q.,https://bina.az//items/2764090,i̇yul 2023,6000000,9 sot,15/18,1500 m²
17211,Bakıxanov q.,https://bina.az//items/3608900,i̇yul 2023,240000,8 sot,3/9,100 m²
17212,Həzi Aslanov m.,https://bina.az//items/3608859,i̇yul 2023,130000,3 otaq,9/9,500 m²


In [231]:
df['rooms'] = df['rooms'].str.replace('otaq', '').str.strip()
df['rooms'] = df['rooms'].str.replace('sot', '').str.strip()
df['rooms'] = df['rooms'].astype(int)
df



,Unvan,Link,Tarix,Qiymet,rooms,floor,area
0,Şağan q.,https://bina.az//items/3495797,bugün 16:06,260000,3,13/16,160 m²
1,Mərdəkan q.,https://bina.az//items/3004266,bugün 16:06,195000,2,10/16,160 m²
2,Neftçilər m.,https://bina.az//items/3632290,bugün 16:05,137000,2,7/9,75 m²
3,Masazır q.,https://bina.az//items/3562385,bugün 16:05,36000,3,7/12,60 m²
4,Qara Qarayev m.,https://bina.az//items/3565113,bugün 16:05,138600,3,10/17,5 m²
...,...,...,...,...,...,...,...
17209,Yasamal r.,https://bina.az//items/3608907,i̇yul 2023,185000,3,7/7,400 m²
17210,Badamdar q.,https://bina.az//items/2764090,i̇yul 2023,6000000,9,15/18,1500 m²
17211,Bakıxanov q.,https://bina.az//items/3608900,i̇yul 2023,240000,8,3/9,100 m²
17212,Həzi Aslanov m.,https://bina.az//items/3608859,i̇yul 2023,130000,3,9/9,500 m²


In [232]:
df['floor'] = df['floor'].str.replace('', '').str.strip()
df['floor'] = df['floor'].str.extract(r'^(\d)', expand=False)
df['floor'] = df['floor'].astype(int)
df

,Unvan,Link,Tarix,Qiymet,rooms,floor,area
0,Şağan q.,https://bina.az//items/3495797,bugün 16:06,260000,3,1,160 m²
1,Mərdəkan q.,https://bina.az//items/3004266,bugün 16:06,195000,2,1,160 m²
2,Neftçilər m.,https://bina.az//items/3632290,bugün 16:05,137000,2,7,75 m²
3,Masazır q.,https://bina.az//items/3562385,bugün 16:05,36000,3,7,60 m²
4,Qara Qarayev m.,https://bina.az//items/3565113,bugün 16:05,138600,3,1,5 m²
...,...,...,...,...,...,...,...
17209,Yasamal r.,https://bina.az//items/3608907,i̇yul 2023,185000,3,7,400 m²
17210,Badamdar q.,https://bina.az//items/2764090,i̇yul 2023,6000000,9,1,1500 m²
17211,Bakıxanov q.,https://bina.az//items/3608900,i̇yul 2023,240000,8,3,100 m²
17212,Həzi Aslanov m.,https://bina.az//items/3608859,i̇yul 2023,130000,3,9,500 m²


In [233]:
df['area'] = df['area'].str.replace('m²', '').str.strip()
df['area'] = df['area'].astype(int)

In [234]:
df.dtypes

Unvan     object
Link      object
Tarix     object
Qiymet     int32
rooms      int32
floor      int32
area       int32
dtype: object

In [249]:
df.isnull().sum()
df1=df.select_dtypes(exclude='object')
df1

,Qiymet,rooms,floor,area
0,260000,3,1,160
1,195000,2,1,160
2,137000,2,7,75
3,36000,3,7,60
4,138600,3,1,5
...,...,...,...,...
17209,185000,3,7,400
17210,6000000,9,1,1500
17211,240000,8,3,100
17212,130000,3,9,500


In [254]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max Normalizasyonu
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df1.values)

# Normalize edilen verileri DataFrame'e dönüştürme
normalized_df = pd.DataFrame(normalized_data, columns=df1.columns)

normalized_df

,Qiymet,rooms,floor,area,real,prd
0,0.007957,0.333333,0.00,0.000578,0.007957,0.011203
1,0.005957,0.222222,0.00,0.000578,0.005957,0.008342
2,0.004172,0.222222,0.75,0.000269,0.004172,0.005790
3,0.001065,0.333333,0.75,0.000215,0.001065,0.001346
4,0.004222,0.333333,0.00,0.000015,0.004222,0.005860
...,...,...,...,...,...,...
17209,0.005649,0.333333,0.75,0.001451,0.005649,0.007902
17210,0.184580,1.000000,0.00,0.005451,0.184580,0.260745
17211,0.007342,0.888889,0.25,0.000360,0.007342,0.010324
17212,0.003957,0.333333,1.00,0.001815,0.003957,0.005482


In [257]:
target='Qiymet'
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into features and target variable
X = normalized_df.drop(target, axis=1)
y = normalized_df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=42)

# Create the Random Forest Regression model
rfg_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rfg_model.fit(X_train, y_train)

# Predict on the test data
y_pred = rfg_model.predict(X_test)
normalized_df['prd']=rfg_model.predict(X)
normalized_df['real']=y
score1=rfg_model.score(X_test,y_test)
score1

0.953859080251638